# Analysis of Organoid Data
Look at maximum entropy models. Inspired by multiple papers, mainly [this](https://www.nature.com/articles/nature04701.pdf).

## Imports


In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns 
import matplotlib as mpl

#Import SpikeSet:
from SpikeSet import spikeset_run_jhu as spikeset_run
from SpikeSet import load_data_jhu as load_data

In [ ]:
save_path='../Functional Connectivity/OrganoidResults/' #CHANGE PATHS HERE
og_save_path='../CSVs/Organoids/'
plot_scale=3 #Scale for plotting (default 1)
dataset=2 #choose dataset

## Load Data and save to CSVs/DataFrames:

In [ ]:
### Manually change the folders to load the data due to the way data was shared

#Hopkins Dataset 1
if dataset == 1:
    og_load_path='../HopkinsData/TimeCourseData/'
    #CHANGE FOLDER NAME BELOW FOR EACH RUN
    folder_name='Raw Data Week 5.5 to 8.5 (run 8 and LD)/'#'Raw Data Week 9.5 to 12.5 (run 6 and 7)/'#'Raw Data Week 5.5 to 8.5 (run 8 and LD)/'#'
    #CHANGE RUN NAME AS NEEDED
    run_name='Run8' #Run7,Run8,Run9
    filenames=['div3','div5','div7','div9','div11','div13','div15','div17','div19','div21']
    # # filename='div3' #change div name (div = days in vitro) 
    load_path_H5 =og_load_path+folder_name+run_name+'/'
    save_path_CSV =og_save_path+folder_name+run_name+'/'
    import os
    if not os.path.exists(save_path_CSV):
        os.makedirs(save_path_CSV)
else:
    #Hopkins Dataset 2
    og_load_path='../HopkinsData2/'
    #CHANGE FOLDER NAME BELOW FOR WELLS 4-6
    folder_name='230601 RUN 8 Wells 1-3/'#230602 RUN 8 Wells 4-6/
    #CHANGE SUBFOLDER NAMES AS NEEDED:
    subfolder_names=['#1 (1-3 baseline)','#10 (well #3 post stim 1)','#11 (well #3 post stim 2)','#12 (well #3 post stim 3)',
                     '#13 (well #3 post stim 4)','#14 (Well #1 180 minutes Well #2 120 minutes Well #3 60 minutes)',
                     '#15 (Well #1 210 minutes Well #2 150 minutes Well #3 90 minutes)',
                     '#16 (Well #1 240 minutes Well #2 180 minutes Well #3 120 minutes)',
                     '#17 (Well #1 310 minutes Well #2 210 minutes Well #3 150 minutes)',
                     '#18 (Well #1 340 minutes Well #2 240 minutes Well #3 180 minutes)',
                     '#2 (well #1 post stim 1)','#3 (well #1 post stim 2)','#4 (well #1 post stim 3)','#5 (well #1 post stim 4)',
                     '#6 (well #2 post stim 1)','#7 (well #2 post stim 2)','#8 (well #2 post stim 3)','#9 (well #2 post stim 4)']
    # filename='div3' #change div name (div = days in vitro) 
    load_path_H5 =og_load_path+folder_name+'/'
    save_path_CSV =og_save_path+folder_name+'/'
    import os
    if not os.path.exists(save_path_CSV):
        os.makedirs(save_path_CSV)

In [ ]:
#Save CSVs:
if dataset == 1:
    for filename in tqdm(filenames):
        dfs=[]
        dfs,df_keys,electrodes,f = load_data.load_h5_to_pd_jhu(load_path_H5+filename+'/data.raw.h5')#'/data.raw.h5') #M05898data, M05912data
        i = 0
        for file in dfs:
            file=file.dropna()
            file.to_csv(save_path+'_well'+str(i+1)+'.csv') #save each well to csv file
            i+=1
else:
    for subfolder in tqdm(subfolder_names):
        dfs=[]
        dfs,df_keys,electrodes,f = load_data.load_h5_to_pd_jhu(load_path_H5+subfolder+'/data.raw.h5')#'/data.raw.h5') #M05898data, M05912data
        i = 0
        for file in dfs:
            file=file.dropna()
            file.to_csv(save_path+subfolder+'_well'+str(i+1)+'.csv') #save each well to csv file
            i+=1  

## Run Organoid Analysis

### Hopkins Dataset 1


In [ ]:
### List of Parameters
runs = [6]
wells = [0,1,2, 3, 4, 5]
divs = [3, 5, 7, 9, 11, 13, 15, 17, 19, 21]
LDnames=['M05898','M05912']

params = {}
params["comment"] = "Add any information that you want"
params["sampling_freq"] = 20000
params["convolution_size"] = int(0.1*params["sampling_freq"])
params["convolution_step"] = int(0.05*params["sampling_freq"])
params["min_electrode_frequency"] = 0
params["permute_columns"] = 0
params["only_binary"] = True
params["seed"] = 1
results_path_default = save_path
params["results_path"] = save_path

## Uncomment for Video:
# video_params = {}
# video_params["grouping_size"] = 500
# video_params["grouping_step"] = 500
# video_params["every_n"] = 3
# # video_params["num_edges"] = 10000
# video_params["exponential_coefficient"] = 1 * 10 ** (-10)
# video_params["use_exp"] = False
# video_params["normalize"] = True
# video_params["distance_metric"] = "mutual_information"

#Set up Results dict
results = {}
results["community_metric"] = np.zeros((len(runs), len(wells), len(divs)))
results["electrode_count"] = np.zeros((len(runs), len(wells), len(divs)))
results["shortest_path_length"] = np.zeros((len(runs), len(wells), len(divs)))
results["clustering_coeff"] = np.zeros((len(runs), len(wells), len(divs)))
results["dist_matrix"] = {}

if runs[0] == 9:
    results["bct_vals"] = [[[[{} for s in range(len(LDnames))] for i in range(len(divs))] for j in range(len(wells[3:]))] for k in range(len(runs))]
    results["dist_matrix"] = [[[[{} for s in range(len(LDnames))] for i in range(len(divs))] for j in range(len(wells[3:]))] for k in range(len(runs))]
    pos = {}
else:
    results["bct_vals"] = [[[{} for i in range(len(divs))] for j in range(len(wells))] for k in range(len(runs))]
    results["dist_matrix"] = [[[{} for i in range(len(divs))] for j in range(len(wells))] for k in range(len(runs))]

    pos = {} #[for i in range(len(wells))] for j in
    
#Generate Full Electrode Array
full_graph,pos=spikeset_run.generate_Electrode_Array(pos)


In [ ]:
#Run Dataset 1 graph analysis
for run_index, run in enumerate(runs): #loop through Run folders
    if run==6 or run == 7: 
        load_path = og_load_path+'/Raw Data Week 9.5 to 12.5 (run 6 and 7)/Run/' 
    else:
        load_path =og_load_path+'/Raw Data Week 5.5 to 8.5 (run 8 and LD)/Run/' 
    
    if run == 9: #Run 9 is separated into two distinct folders so we need to halve the wells tracker (vals)
        vals = wells[3:]
    else:
        vals = wells
    for well_index, well in tqdm(enumerate(vals)):  #Loop through wells using well tracker 
        print('\n\n WELL '+str(well)+'\n\n')
        for div_index, div in enumerate(divs): #loop through days in vitro
#             try:
            if run == 9: 
                ldcount=0 
                for LDname in LDnames: #loop through two folders for Run 9 only
                    if LDname == 'M05912':
                        path=  load_path + str(run) + "/div" + str(div) + "well" + str(well) + "_M05912.csv"
                    elif LDname == 'M05898':
                            path=  load_path + str(run) + "/div" + str(div) + "well" + str(well) + "_M05898.csv"
                    params["run"] = run
                    params["results_path"] = results_path_default + "Run_" + str(run) + "/well_" + str(well) + "/" + "div_" + str(div)+"/"+LDname
                    results=spikeset_run.run_dataset1(run,run_index,well,well_index,div,div_index,path,ldcount,params,results,pos,full_graph)
                    ldcount+=1
            else: #All other runs don't need looping through
                params["run"] = run
                params["results_path"] = results_path_default + "Run_" + str(run) + "/well_" + str(well) + "/" + "div_" + str(div)+"/"
                path=  load_path + str(run) + "/div" + str(div) + "well" + str(well) + ".csv"
                results=spikeset_run.run_dataset1(run,run_index,well,well_index,div,div_index,path,-1,params,results,pos,full_graph)
                print(results["bct_vals"][run_index][well_index][div_index]["num_nodes"])
#             except:
#                 print('error')
#                 continue
        ## Save Movie if wanted:
          # spike_set.SpikeSet_ConnectivityMovie(video_params)


# Save results dict as pickle:         
import pickle
with open(results_path_default+"/"+str(run)+"_results.pkl", "wb") as f:
    pickle.dump(results,f)

### Hopkins Dataset 2


In [ ]:
### List of Parameters

runs = [1,2]    
wells = [0,1,2, 3, 4, 5]
recording_num=list(range(18))

params = {}
params["comment"] = "Add any information that you want"
params["sampling_freq"] = 20000
params["convolution_size"] = int(0.1*params["sampling_freq"])
params["convolution_step"] = int(0.05*params["sampling_freq"])
params["min_electrode_frequency"] = 0
params["permute_columns"] = 0
params["only_binary"] = True
params["seed"] = 1
results_path_default = save_path
params["results_path"] = save_path

## Uncomment for video
# video_params = {}
# video_params["grouping_size"] = 500
# video_params["grouping_step"] = 500
# video_params["every_n"] = 3
# video_params["num_edges"] = 200
# video_params["exponential_coefficient"] = 1 * 10 ** (-10)
# video_params["use_exp"] = False
# video_params["normalize"] = True
# video_params["distance_metric"] = "mutual_information"
#Set up Results dict



results = {}
results["dist_matrix"] = {}

#Set distance matrix data structure
for run in runs:
    results["dist_matrix"][run-1] = {}
    for well in wells:
        results["dist_matrix"][run-1][well] = {}
        for recording in recording_num:
            results["dist_matrix"][run-1][well][recording] = {}

results["bct_vals"] = [[[{} for i in range(len(recording_num))] for j in range(len(wells))] for k in range(len(runs))]
pos = {} 

#Generate Full Graph
full_graph,pos=spikeset_run.generate_Electrode_Array(pos)

In [ ]:
#Run Dataset 2 graph analysis


load_paths=[og_load_path+'/230601 RUN 8 Wells 1-3/',og_load_path+'/230602 RUN 8 Wells 4-6/']

for run in runs:
    if run == 0:
        load_path=load_paths[0]
    else:
        load_path=load_paths[1]
    for file in tqdm(os.listdir(load_path)):
        recording_num1=int(file[1:3])
        well_num=int(file.split(os.extsep)[0][-1:])
        params["results_path"] = results_path_default + "Run_8" + "/well_" + str(well_num) + "_Data2/" + "folder_"+str(run)+"_recording_" + str(recording_num1)+"/"
        path=  load_path + file
        results=spikeset_run.run_dataset2(run-1,well_num,well_num-1,recording_num1-1,load_path,path,params,results,pos,full_graph)

      # spike_set.SpikeSet_ConnectivityMovie(video_params)


# import pickle
# with open(results_path_default+"/"+str(8)+"_Dataset2_results.pkl", "wb") as f:
#     pickle.dump(results,f)

#### IF YOU JUST WANT TO CHECK THE RAW DATA FOR CHANNELS, YOU CAN DO THAT  HERE


In [ ]:
# load_paths=['../CSVs/Organoids/230601 RUN 8 Wells 1-3/','../CSVs/Organoids/230602 RUN 8 Wells 4-6/']
# load_path=load_paths[0]
# for file in tqdm(os.listdir(load_path)):
#     recording_num1=int(file[1:3])
#     well_num=int(file.split(os.extsep)[0][-1:])
#     params["results_path"] = results_path_default + "Run_8" + "/well_" + str(1) + "_Data2/" + "folder_"+str(1)+"_recording_" + str(1)+"/"
#     path=  load_path + file
#     data=pd.read_csv(path)
#     data = pd.DataFrame(data=data)

## Graph Theory Analysis

### Hopkins Dataset 1


In [ ]:
#Load saved pkl files

runs=[8,9] #8 represents runs 6-8, 9 represents run 9
results_path_default = '../Functional Connectivity/OrganoidResults/'
for run in runs:
    with open(results_path_default+"/"+str(run)+"_results.pkl", "rb") as f:
        tmp = (pickle.load(f))
        if run == 9:
            bct_results9 = ((tmp['bct_vals'][0]))
            results9     = ((tmp['dist_matrix']))
        else:
            bct_results  = ((tmp['bct_vals']))
            results      = ((tmp['dist_matrix']))

In [ ]:
"""
Highly innefficient way to combine results of all runs/wells/divs into one large array for each graph theory metric.
This was done because of the way the data was provided. 
"""

runs=[6,7,8,9]

#wells,divs,ld
vals       = wells[3:]
num_nodes9 = np.zeros((len(vals),len(divs),len(LDnames)))
num_edges9 = np.zeros((len(vals),len(divs),len(LDnames)))
density9   = np.zeros((len(vals),len(divs),len(LDnames)))
degree9    = np.zeros((len(vals),len(divs),len(LDnames)))
q9         = np.zeros((len(vals),len(divs),len(LDnames)))
pcoeff9    = [[[[] for s in range(len(LDnames))] for i in range(len(divs))] for j in range(len(vals))]
mz9        = [[[[] for s in range(len(LDnames))] for i in range(len(divs))] for j in range(len(vals))]
avg_spl9   = np.zeros((len(vals),len(divs),len(LDnames)))
pos9       = [[[[] for s in range(len(LDnames))] for i in range(len(divs))] for j in range(len(vals))]
weights9   = [[[[] for s in range(len(LDnames))] for i in range(len(divs))] for j in range(len(vals))]
 
#wells,divs
vals       = wells
num_nodes  = np.zeros((len(runs),len(vals),len(divs)))
num_edges  = np.zeros((len(runs),len(vals),len(divs)))
density    = np.zeros((len(runs),len(vals),len(divs)))
degree     = np.zeros((len(runs),len(vals),len(divs)))
q          = np.zeros((len(runs),len(vals),len(divs)))
pcoeff     = [[[[] for s in range(len(divs))] for i in range(len(vals))] for j in range(len(runs))]
mz         = [[[[] for s in range(len(divs))] for i in range(len(vals))] for j in range(len(runs))]
avg_spl    = np.zeros((len(runs),len(vals),len(divs)))
pos8       = [[[[] for s in range(len(divs))] for i in range(len(vals))] for j in range(len(runs))]
weights    = [[[[] for s in range(len(divs))] for i in range(len(vals))] for j in range(len(runs))]

for run_index, run in tqdm(enumerate(runs)):
    if run == 9:
        vals = wells[3:]
    else:
        vals = wells
    for well_index, well in tqdm(enumerate(vals)):    
        for div_index, div in enumerate(divs):
            if run == 9:
                ldcount=0
                for LDname in LDnames:
                    if well == 5 and div ==17:
                        num_nodes9[well_index,div_index,ldcount] = np.nan
                        num_edges9[well_index,div_index,ldcount] = np.nan
                        density9[well_index,div_index,ldcount]   = np.nan
                        degree9[well_index,div_index,ldcount]    = np.nan
                        q9[well_index,div_index,ldcount]         = np.nan
                        pcoeff9[well_index][div_index][ldcount]  = [np.nan]*13209
                        mz9[well_index][div_index][ldcount]      = [np.nan]*13209
                        avg_spl9[well_index,div_index,ldcount]   = np.nan
                        pos9[well_index][div_index][ldcount]     = [np.nan]*13209
                        weights9[well_index][div_index][ldcount] = np.nan

                    else:
                        num_nodes9[well_index,div_index,ldcount] = bct_results9[well_index][div_index][ldcount]['num_nodes']
                        num_edges9[well_index,div_index,ldcount] = bct_results9[well_index][div_index][ldcount]['num_edges']
                        density9[well_index,div_index,ldcount]   = bct_results9[well_index][div_index][ldcount]['density']
                        degree9[well_index,div_index,ldcount]    = bct_results9[well_index][div_index][ldcount]['degree'][0]
                        q9[well_index,div_index,ldcount]         = bct_results9[well_index][div_index][ldcount]['community_metric']
                        pcoeff9[well_index][div_index][ldcount]  = bct_results9[well_index][div_index][ldcount]['pcoeff']
                        mz9[well_index][div_index][ldcount]      = bct_results9[well_index][div_index][ldcount]['mz']
                        avg_spl9[well_index,div_index,ldcount]   = bct_results9[well_index][div_index][ldcount]['avg spl']
                        pos9[well_index][div_index][ldcount]     = list(bct_results9[well_index][div_index][ldcount]['nw_pos'].values())
                        weights9[well_index][div_index][ldcount] = results9[0][well_index][div_index][ldcount]

                    ldcount+=1      
            else:
                num_nodes[run_index,well_index,div_index]        = bct_results[run_index][well_index][div_index]['num_nodes']
                num_edges[run_index,well_index,div_index]        = bct_results[run_index][well_index][div_index]['num_edges']
                density[run_index,well_index,div_index]          = bct_results[run_index][well_index][div_index]['density']
                degree[run_index,well_index,div_index]           = bct_results[run_index][well_index][div_index]['degree'][0]
                q[run_index,well_index,div_index]                = bct_results[run_index][well_index][div_index]['community_metric']
                pcoeff[run_index][well_index][div_index]         = bct_results[run_index][well_index][div_index]['pcoeff']
                mz[run_index][well_index][div_index]             = bct_results[run_index][well_index][div_index]['mz']
                avg_spl[run_index,well_index,div_index]          = bct_results[run_index][well_index][div_index]['avg spl']
                pos8[run_index][well_index][div_index]           = list(bct_results[run_index][well_index][div_index]['nw_pos'].values())
                weights[run_index][well_index][div_index]        = results[run][well][div]

if run == 9: #combine M0898 and M0912:
    num_nodes9 = np.vstack((num_nodes9[:,:,0],(num_nodes9[:,:,1])))
    num_edges9 = np.vstack((num_edges9[:,:,0],(num_edges9[:,:,1])))
    density9   = np.vstack((density9[:,:,0],(density9[:,:,1])))
    degree9    = np.vstack((degree9[:,:,0],(degree9[:,:,1])))
    q9         = np.vstack((q9[:,:,0],(q9[:,:,1])))
    avg_spl9   = np.vstack((avg_spl9[:,:,0],(avg_spl9[:,:,1])))

    a=[];b1=[];a2=[];a3=[]
    b=[];a1=[];b2=[];b3=[]
    for i in range(len(pcoeff9)):
        a.append(pd.DataFrame(pcoeff9[i]).values[:,0])
        b.append(pd.DataFrame(pcoeff9[i]).values[:,1])
        a1.append(pd.DataFrame(mz9[i]).values[:,0])
        b1.append(pd.DataFrame(mz9[i]).values[:,1])
        a2.append(pd.DataFrame(pos9[i]).values[:,0])
        b2.append(pd.DataFrame(pos9[i]).values[:,1])
        a3.append(pd.DataFrame(weights9[i]).values[:,0])
        b3.append(pd.DataFrame(weights9[i]).values[:,1])
    pcoeff9  = np.vstack((np.array(a), np.array(b)))
    mz9      = np.vstack((np.array(a1), np.array(b1)))
    pos9     = np.vstack((np.array(a2), np.array(b2)))
    weights9 = np.vstack((np.array(a3), np.array(b3)))
#Combine run 9 with others: 
#Check this
num_nodes[-1]=num_nodes9
num_edges[-1]=num_edges9
density[-1]=density9
degree[-1]=degree9
q[-1]=q9
avg_spl[-1]=avg_spl9
for i in range(len(pcoeff9)):
    for j in range(len(pcoeff9[i])):
        pcoeff[-1][i][j]  = pcoeff9[i,j] #[run, well, div, nodes]
        mz[-1][i][j]      = pcoeff9[i,j]
        pos8[-1][i][j]    = pos9[i,j]
        weights[-1][i][j] = weights9[i,j]

In [ ]:
#weird data type fixing: some data is saved in a strange data type with some graph theory toolboxes, so need to resave it as a float
pcoeff_new = [None]*len(pcoeff9)
mz_new     = [None]*len(mz9)
for i in range(len(pcoeff9)):
    
    tmp  = np.vstack(pcoeff9[i][:]).astype(float)
    tmp2 = np.vstack(mz9[i][:]).astype(float)

    pcoeff_new[i] = tmp
    mz_new[i]     = tmp2

pcoeff_new = np.array(pcoeff_new)
mz_new     = np.array(mz_new)

In [ ]:
#Combine weeks:
num_nodes_week12 = np.mean(num_nodes[0:2],axis=0)
num_nodes_week8  = np.nanmean((num_nodes[2],num_nodes9),axis=0)

num_edges_week12 = np.mean(num_edges[0:2],axis=0)
num_edges_week8  = np.nanmean((num_edges[2],num_edges9),axis=0)

q_week12         = np.mean(q[0:2],axis=0)
q_week8          = np.nanmean((q[2],q9),axis=0)

avg_spl_week12   = np.mean(q[0:2],axis=0)
avg_spl_week8    = np.nanmean((q[2],q9),axis=0)

pcoeff_week12    = np.nanmean(pcoeff[0:2],axis=0)
pcoeff_week8     = np.nanmean(np.stack((pcoeff_new,pcoeff[2])),axis=0)

mz_week12        = np.nanmean(mz[0:2],axis=0)
mz_week8         = np.nanmean(np.stack((mz_new,mz[2])),axis=0)

In [ ]:
#Results Per DIV, across WELLS
#ALL RUNS:

mpl.rcParams['pdf.fonttype'] = 42

fig,ax=plt.subplots(1,3,figsize=(20,4))

#Num Nodes
b = ax[0].plot(np.nanmean(num_nodes_week8, axis=0), c='#03045e', label='Week 6-9')
a = ax[0].plot(np.nanmean(num_nodes_week12, axis=0), c='#e63946', label='Week 10-13')

ax[0].fill_between(range(len(np.nanmean(num_nodes_week12,axis=0))), np.nanmean(num_nodes_week12,axis=0)+np.nanstd(num_nodes_week12,axis=0), np.nanmean(num_nodes_week12,axis=0)-np.nanstd(num_nodes_week12,axis=0), alpha=0.2, color='#e63946')
ax[0].fill_between(range(len(np.nanmean(num_nodes_week8,axis=0))), np.nanmean(num_nodes_week8,axis=0)+np.nanstd(num_nodes_week8,axis=0), np.nanmean(num_nodes_week8,axis=0)-np.nanstd(num_nodes_week8,axis=0), alpha=0.2, color='#03045e')

ax[0].set_xticks(range(len(divs)))
ax[0].set_xticklabels(divs)
ax[0].set_xlabel('DOM')
ax[0].set_ylabel('Avg Number of Nodes')
ax[0].legend(title='Age of bMPS',ncols=2)#['Run 6','Run 7','Run 8','Run 9'])

# #Num Edges
a = ax[1].plot(np.nanmean(num_edges_week12, axis=0), c='#e63946', label='Week 10-13')
b = ax[1].plot(np.nanmean(num_edges_week8, axis=0), c='#03045e', label='Week 6-9')

ax[1].fill_between(range(len(np.nanmean(num_edges_week12,axis=0))),np.nanmean(num_edges_week12,axis=0)+np.nanstd(num_edges_week12,axis=0),np.nanmean(num_edges_week12,axis=0)-np.nanstd(num_edges_week12,axis=0),alpha=0.2,color='#e63946')
ax[1].fill_between(range(len(np.nanmean(num_edges_week8,axis=0))),np.nanmean(num_edges_week8,axis=0)+np.nanstd(num_edges_week8,axis=0),np.nanmean(num_edges_week8,axis=0)-np.nanstd(num_edges_week8,axis=0),alpha=0.2,color='#03045e')

ax[1].set_xticks(range(len(divs)))
ax[1].set_xticklabels(divs)
ax[1].set_xlabel('DOM')
ax[1].set_ylabel('Avg Number of Edges')

# #modularity
a = ax[2].plot(np.nanmean(q_week12, axis=0), c='#e63946', label='Week 10-13')
b = ax[2].plot(np.nanmean(q_week8, axis=0), c='#03045e', label='Week 6-9')

ax[2].fill_between(range(len(np.nanmean(q_week12,axis=0))),np.nanmean(q_week12,axis=0)+np.nanstd(q_week12,axis=0),np.nanmean(q_week12,axis=0)-np.nanstd(q_week12,axis=0),alpha=0.2,color='#e63946')
ax[2].fill_between(range(len(np.nanmean(q_week8,axis=0))),np.nanmean(q_week8,axis=0)+np.nanstd(q_week8,axis=0),np.nanmean(q_week8,axis=0)-np.nanstd(q_week8,axis=0),alpha=0.2,color='#03045e')

ax[2].set_xticks(range(len(divs)))
ax[2].set_xticklabels(divs)
ax[2].set_xlabel('DOM')
ax[2].set_ylabel('Modularity (q)')

#Save PDF
fig.savefig('../Functional Connectivity/OrganoidResults/AllRuns.pdf',format='pdf',dpi=300)

In [ ]:
#Draw Pos + PCoeff/MZ graphs:
labels=['Week 12','Week 8']
nodes=np.array([i for i in full_graph.nodes])
for i in tqdm(range(2)):
    for j in range(len(pcoeff_week12)):
        for k in range(len(pcoeff_week12[j])):
            fig,axes=plt.subplots(1,2,figsize=(10,4))
            ax=axes.flat
            for x in range(len(ax)):
                nx.draw(full_graph,pos,node_color='xkcd:pale orange',node_size=2,alpha=0.01,ax=ax[x])
            if i == 0:
                nx.draw(full_graph,pos,node_color=pcoeff_week12[j][k],cmap=plt.cm.Reds,node_size=5,ax=ax[0])
                nx.draw(full_graph,pos,node_color=mz_week12[j][k],cmap=plt.cm.Blues,node_size=5,ax=ax[1])
            else:
                nx.draw(full_graph,pos,node_color=pcoeff_week8[j][k],cmap=plt.cm.Reds,node_size=5,ax=ax[0])
                nx.draw(full_graph,pos,node_color=mz_week8[j][k],cmap=plt.cm.Blues,node_size=5,ax=ax[1])
            plt.savefig(results_path_default+'Summary Results/pcoeff_mz_'+labels[i]+'_well '+str(wells[j])+'_div '+str(divs[k])+'.png',dpi=300, bbox_inches='tight')

### Hopkins Dataset 2

In [ ]:
"""
Repeat Graph theory Analysis but with the second dataset
"""

results_path_default = save_path

with open(results_path_default+"/8_Dataset2_results.pkl", "rb") as f:
    tmp=(pickle.load(f))
    bct_results1=((tmp['bct_vals']))
    results1=((tmp['dist_matrix']))

bct_results=np.transpose(np.array(bct_results1),axes=[1,2,0])

In [ ]:
### Only have 2 runs this time so it's easier to set up and loop through.
runs = [1,2]    
wells = [0,1,2, 3, 4, 5]
recording_num=list(range(18))

combined_bct_results = [[{} for i in range(len(recording_num)*2)] for j in range(len(wells))]
combined_results = [[{} for i in range(len(recording_num)*2)] for j in range(len(wells))]

runs = [1,2]
for well_index, well in tqdm(enumerate(wells)):    
    i=0
    for recording in recording_num:
        for run_index, run in enumerate(runs):
            combined_bct_results[well_index][i]=bct_results[well_index][recording][run_index]
            combined_results[well_index][i]=results1[run_index][well_index][recording]

            i+=1

In [ ]:
# Set of relevant channels defined by Dowlette and Sam
relevant_chans=[[195,320,422,372,674,110],
               [426,205,239,108,429,462,526,467,247,484,534,146,329,129],
               [424,193,463,9,181,102,159,294,378,235,435,272,75,25,436,317,107,275,471,211,400,303,440,19,404,469,146,99,363],
               [659,73,834,981,704,687,466,455,451,301],
               [537,548,656,716,179,975,502,961,363,499,685,590,822,586,651,806],
               [147,184,603,789,360,249,569,948,258,862,8,734,759,435,247,439,419,573,105,815,289,285,50,916,206,969,984,397,571,979,157,450,380,531,415,658,765,517,961,92]]


In [ ]:
#wells,divs
vals               = wells
num_nodes          = np.zeros((len(vals),len(recording_num)*2))
num_edges          = np.zeros((len(vals),len(recording_num)*2))
density            = np.zeros((len(vals),len(recording_num)*2))
degree             = np.zeros((len(vals),len(recording_num)*2))
q                  = np.zeros((len(vals),len(recording_num)*2))
pcoeff             = [[[] for s in range(len(recording_num)*2)] for i in range(len(vals))]
mz                 = [[[] for s in range(len(recording_num)*2)] for i in range(len(vals))]
avg_spl            = np.zeros((len(vals),len(recording_num)*2))
relevant_elecs_xy  = [[[] for s in range(len(recording_num)*2)] for i in range(len(vals))]
relevant_elecs     = [[[] for s in range(len(recording_num)*2)] for i in range(len(vals))]
relevant_elecs_pos = [[[] for s in range(len(recording_num)*2)] for i in range(len(vals))]
relevant_adjMat    = [[[] for s in range(len(recording_num)*2)] for i in range(len(vals))]

for well_index, well in tqdm(enumerate(vals)):    
    for i in range(len(recording_num)*2):
        num_nodes[well_index,i]          = combined_bct_results[well_index][i]['num_nodes']
        num_edges[well_index,i]          = combined_bct_results[well_index][i]['num_edges']
        density[well_index,i]            = combined_bct_results[well_index][i]['density']
        degree[well_index,i]             = combined_bct_results[well_index][i]['degree'][0]
        q[well_index,i]                  = combined_bct_results[well_index][i]['community_metric']
        pcoeff[well_index][i]            = combined_bct_results[well_index][i]['pcoeff']
        mz[well_index][i]                = combined_bct_results[well_index][i]['mz']
        avg_spl[well_index,i]            = combined_bct_results[well_index][i]['avg spl']
        chan_idx                         = np.where(np.in1d(combined_bct_results[well_index][i]['channel_ids'],relevant_chans[well_index]))[0]
        relevant_elecs[well_index][i]    = chan_idx
        relevant_elecs_xy[well_index][i] = np.array(list(combined_bct_results[well_index][i]['nw_pos'].values()))[chan_idx]
        relevant_adjMat[well_index][i]   = combined_results[well_index][i][chan_idx,:][:,chan_idx]
        #Relevant Pos
        k = 0
        a = {}
        for j in relevant_elecs[well_index][i]:
            a.update({k:tuple(relevant_elecs_xy[well_index][i][k])})
            k+=1
        relevant_elecs_pos[well_index][i] = a

In [ ]:
#Plot functional connectivity + PCoeff/MZ graphs:
nodes = np.array([i for i in full_graph.nodes])
for j in range(len(relevant_adjMat[0])):
        fig = plt.figure()
        ax  = plt.gca()
        G   = nx.from_numpy_array(relevant_adjMat[0][j])
        nx.draw(full_graph,pos,node_color='xkcd:pale orange',node_size=2,alpha=0.05,ax=ax)
        nx.draw_networkx_nodes(G,relevant_elecs_pos[0][j],node_size=2,ax=ax)#,node_color=pcoeff[0][j],cmap=plt.cm.Reds,node_size=5,ax=ax[0])

In [ ]:
#Results Per DIV, across WELLS
#ALL RUNS:

fig,ax = plt.subplots(1,3,figsize=(20,4))
fig.suptitle('All Runs')
#Degree
colors = ['r','orange','yellow','g','cyan','b']
for i in range(len(num_nodes)):
    ax[0].plot(num_nodes[i,:],c=colors[i])

ax[0].set_xticks(range(36))
ax[0].set_xticklabels(range(1,37),rotation=45)
ax[0].set_xlabel('Folder #')
ax[0].set_ylabel('Avg Number of Nodes')
ax[0].legend(['Well 1','Well 2','Well 3','Well 4','Well 5','Well 6'])

#edges
for i in range(len(num_edges)):
    ax[1].plot(num_edges[i,:],c=colors[i])

ax[1].set_xticks(range(36))
ax[1].set_xticklabels(range(1,37),rotation=45)
ax[1].set_xlabel('Folder #')
ax[1].set_ylabel('Number of Edges')
ax[1].legend(['Well 1','Well 2','Well 3','Well 4','Well 5','Well 6'])

#modularity
for i in range(len(q)):
    ax[2].plot(q[i,:],c=colors[i])

ax[2].set_xticks(range(36))
ax[2].set_xticklabels(range(1,37),rotation=45)
ax[2].set_xlabel('Folder #')
ax[2].set_ylabel('Modularity')
ax[2].legend(['Well 1','Well 2','Well 3','Well 4','Well 5','Well 6'])


In [ ]:
#Seperate into before, during, after stimulation:
import copy

wellNames = ['Well1','Well2','Well3','Well4','Well5','Well6']
before = {'Number of Nodes':[np.nan]*len(recording_num),'Number of Edges':[np.nan]*len(recording_num),'Density':[np.nan]*len(recording_num),
        'Degree':[np.nan]*len(recording_num),'Modularity':[np.nan]*len(recording_num),'Participation Coefficient':[np.nan]*len(recording_num),
        'Module Z Score':[np.nan]*len(recording_num),'Shortest Path Length':[np.nan]*len(recording_num)}
during = {'Number of Nodes':[np.nan]*len(recording_num),'Number of Edges':[np.nan]*len(recording_num),'Density':[np.nan]*len(recording_num),
        'Degree':[np.nan]*len(recording_num),'Modularity':[np.nan]*len(recording_num),'Participation Coefficient':[np.nan]*len(recording_num),
        'Module Z Score':[np.nan]*len(recording_num),'Shortest Path Length':[np.nan]*len(recording_num)}
after  = {'Number of Nodes':[np.nan]*len(recording_num),'Number of Edges':[np.nan]*len(recording_num),'Density':[np.nan]*len(recording_num),
        'Degree':[np.nan]*len(recording_num),'Modularity':[np.nan]*len(recording_num),'Participation Coefficient':[np.nan]*len(recording_num),
        'Module Z Score':[np.nan]*len(recording_num),'Shortest Path Length':[np.nan]*len(recording_num)}

wells  = {'Well1':{'before':copy.deepcopy(before),'during':copy.deepcopy(during),'after':copy.deepcopy(after)},
       'Well2':{'before':copy.deepcopy(before),'during':copy.deepcopy(during),'after':copy.deepcopy(after)},       
       'Well3':{'before':copy.deepcopy(before),'during':copy.deepcopy(during),'after':copy.deepcopy(after)},  
       'Well4':{'before':copy.deepcopy(before),'during':copy.deepcopy(during),'after':copy.deepcopy(after)},       
       'Well5':{'before':copy.deepcopy(before),'during':copy.deepcopy(during),'after':copy.deepcopy(after)},       
       'Well6':{'before':copy.deepcopy(before),'during':copy.deepcopy(during),'after':copy.deepcopy(after)}}   
j = 0
for well in wells: #each well
    for i in range(len(recording_num)): #each folder
        if j < 3: 
            run_index = 0
        else:
            run_index = 1
        if j == 0 or j == 3: #if well 1 or well 4
            if i == 0: #if recording 1
                wells[well]['before']['Number of Nodes'][i] = bct_results[j][i][run_index]['num_nodes']
                wells[well]['before']['Number of Edges'][i] = bct_results[j][i][run_index]['num_edges']
                wells[well]['before']['Modularity'][i]      = bct_results[j][i][run_index]['community_metric']
            elif i >=1 and i <5: #if recording 2-5
                wells[well]['during']['Number of Nodes'][i] = bct_results[j][i][run_index]['num_nodes']
                wells[well]['during']['Number of Edges'][i] = bct_results[j][i][run_index]['num_edges']
                wells[well]['during']['Modularity'][i]      = bct_results[j][i][run_index]['community_metric']

            elif i >=5 and i <14: #if recording 6-14
                wells[well]['after']['Number of Nodes'][i]  = bct_results[j][i][run_index]['num_nodes']
                wells[well]['after']['Number of Edges'][i]  = bct_results[j][i][run_index]['num_edges']
                wells[well]['after']['Modularity'][i]       = bct_results[j][i][run_index]['community_metric']

        if j == 1 or j == 4: #if well 2 or well 5
            if i == 4:
                wells[well]['before']['Number of Nodes'][i] = bct_results[j][i][run_index]['num_nodes']
                wells[well]['before']['Number of Edges'][i] = bct_results[j][i][run_index]['num_edges']
                wells[well]['before']['Modularity'][i]      = bct_results[j][i][run_index]['community_metric']
            elif i >=5 and i <9:
                wells[well]['during']['Number of Nodes'][i] = bct_results[j][i][run_index]['num_nodes']
                wells[well]['during']['Number of Edges'][i] = bct_results[j][i][run_index]['num_edges']
                wells[well]['during']['Modularity'][i]      = bct_results[j][i][run_index]['community_metric']

            elif i >=9 and i < 16:
                wells[well]['after']['Number of Nodes'][i]  = bct_results[j][i][run_index]['num_nodes']
                wells[well]['after']['Number of Edges'][i]  = bct_results[j][i][run_index]['num_edges']
                wells[well]['after']['Modularity'][i]       = bct_results[j][i][run_index]['community_metric']

        if j == 2 or j == 5: #if well 3 or well 6
            if i == 8:
                wells[well]['before']['Number of Nodes'][i]  = bct_results[j][i][run_index]['num_nodes']
                wells[well]['before']['Number of Edges'][i]  = bct_results[j][i][run_index]['num_edges']
                wells[well]['before']['Modularity'][i]       = bct_results[j][i][run_index]['community_metric']

            elif i >= 8 and i <13:
                wells[well]['during']['Number of Nodes'][i]  = bct_results[j][i][run_index]['num_nodes']
                wells[well]['during']['Number of Edges'][i]  = bct_results[j][i][run_index]['num_edges']
                wells[well]['during']['Modularity'][i]       = bct_results[j][i][run_index]['community_metric']

            elif i >= 13 and i <18:
                wells[well]['after']['Number of Nodes'][i]   = bct_results[j][i][run_index]['num_nodes']
                wells[well]['after']['Number of Edges'][i]   = bct_results[j][i][run_index]['num_edges']
                wells[well]['after']['Modularity'][i]        = bct_results[j][i][run_index]['community_metric']
    j+=1

In [ ]:
#Visualise Before, During, After 

#Results Per DIV, across WELLS
#ALL RUNS:

fig, ax   = plt.subplots(1,3,figsize=(20,5))
#Degree
num_nodes = [[None]*3 for i in range(len(wells))]
i=0
for well in wells:
    num_nodes[i][0] = np.nanmean(wells[well]['before']['Number of Nodes'])
    num_nodes[i][1] = np.nanmean(wells[well]['during']['Number of Nodes'])
    num_nodes[i][2] = np.nanmean(wells[well]['after']['Number of Nodes'])
    i+=1
ax[0].hist(np.array(num_nodes).T)
ax[0].set_xticks([0,1,2])
ax[0].set_xticklabels(['before','during','after'],rotation=45)
ax[0].set_ylabel('Number of Nodes')

# #edges
num_edges = [[None]*3 for i in range(len(wells))]
i = 0
for well in wells:
    num_edges[i][0] = np.nanmean(wells[well]['before']['Number of Edges'])
    num_edges[i][1] = np.nanmean(wells[well]['during']['Number of Edges'])
    num_edges[i][2] = np.nanmean(wells[well]['after']['Number of Edges'])
    i+=1
ax[1].plot(np.array(num_edges).T,'-o')
ax[1].set_xticks([0,1,2])
ax[1].set_xticklabels(['before','during','after'],rotation=45)
ax[1].set_ylabel('Number of Edges')

# #modularity
q = [[None]*3 for i in range(len(wells))]
i = 0
for well in wells:
    q[i][0] = np.nanmean(wells[well]['before']['Modularity'])
    q[i][1] = np.nanmean(wells[well]['during']['Modularity'])
    q[i][2] = np.nanmean(wells[well]['after']['Modularity'])
    i+=1
ax[2].plot(np.array(q).T,'-o')
ax[2].set_xticks([0,1,2])
ax[2].set_xticklabels(['before','during','after'],rotation=45)
ax[2].set_ylabel('Modularity (q)')
ax[2].legend(['Well 1','Well 2','Well 3','Well 4','Well 5','Well 6'])

fig.savefig('../Functional Connectivity/OrganoidResults/AllRuns.pdf',format='pdf',dpi=300)

In [ ]:

# Get the values of num_nodes
num_nodes_values = np.array(num_nodes)
num_nodes        = pd.DataFrame({'before':num_nodes_values[:,0],'during':num_nodes_values[:,1],'after':num_nodes_values[:,2]})

num_edges_values = np.array(num_edges)
num_edges        = pd.DataFrame({'before':num_edges_values[:,0],'during':num_edges_values[:,1],'after':num_edges_values[:,2]})

q_values         = np.array(q)
q                = pd.DataFrame({'before':q_values[:,0],'during':q_values[:,1],'after':q_values[:,2]})

vals             = [num_nodes,num_edges,q]
labels           = ['Num Nodes','Num Edges','Modularity']

In [ ]:
# Create boxplots:

sns.set(style="darkgrid")
sns.set(font_scale=1.4)

# Define custom color palette
custom_palette = sns.color_palette("Set2")
# Swap colors for 'during' and 'after'
custom_palette[1], custom_palette[2] = custom_palette[2], custom_palette[1]

fig, axes = plt.subplots(1,3,figsize=(15, 5.5))
fig.tight_layout(pad=3)
for i,val in enumerate(vals):
    ax = axes.flat[i]
    sns.boxplot(data = val, palette = custom_palette, width = 0.4, showfliers = False, showmeans=True, 
                meanprops = {"markerfacecolor": "black", 
                            "markeredgecolor": "black",
                            "markersize": "5"}, dodge=False, ax=ax)
    ax.legend([], [], frameon=False)

    # Set plot title and labels
    plt.title('Data Frame')
    plt.xlabel('')
    plt.ylabel('Y-axis')

    # Bold x and y tick labels
    ax.set_xticklabels(ax.get_xticklabels(), fontweight='bold')
    ax.set_yticklabels(ax.get_yticklabels(), fontweight='bold')
    
    ax.set_ylabel(labels[i], fontsize=18)
    ax.set_xlabel('Recording Phase', fontsize=18)
    ax.grid(False)
    sns.set(rc={'figure.figsize':(5,4)})

    dfm_mean = val.mean()
    sns.lineplot(data=dfm_mean, marker='o', ax=ax, legend=False,color = 'r')

    # Format y-axis ticks with scaled notation
    yticks = ax.get_yticks()
    if max(yticks) > 10000:
        formatter = ScalarFormatter(useMathText=True)
        formatter.set_scientific(True)
        formatter.set_powerlimits((-1,1))  # Adjust the power limits as needed
        ax.yaxis.set_major_formatter(formatter)

fig.savefig('../OrganoidResults/AllRuns2.pdf',format='pdf',dpi=300)